# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Reviews_Analyzed.csv to Reviews_Analyzed.csv


In [ ]:
import pandas as pd
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import nltk
from nltk.corpus import stopwords

# Load the preprocessed data from the CSV file
df = pd.read_csv('Reviews_Analyzed.csv')
corpus = df['Cleaned_Review'].tolist()

# Tokenize the text
corpus = [doc.split() for doc in corpus]

# Create a dictionary and document-term matrix
dictionary = Dictionary(corpus)
corpus = [dictionary.doc2bow(doc) for doc in corpus]

# Find the optimal number of topics (K) using coherence scores
coherence_scores = []
for k in range(2, 10):
    lda_model = LdaModel(corpus, num_topics=k, id2word=dictionary, passes=15)
    coherence_model = CoherenceModel(model=lda_model, texts=corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append((k, coherence_score))

# Find the K with the highest coherence score
optimal_k = max(coherence_scores, key=lambda x: x[1])[0]

# Train the final LDA model with the selected K
lda_model = LdaModel(corpus, num_topics=optimal_k, id2word=dictionary, passes=15)

# Print and summarize the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Print the selected K and its coherence score
print(f"Optimal K: {optimal_k}")


/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


(0, '0.025*"i" + 0.016*"black" + 0.014*"movie" + 0.013*"marvel" + 0.013*"phase" + 0.011*"wakanda" + 0.010*"the" + 0.010*"really" + 0.009*"would" + 0.009*"film"')
(1, '0.034*"i" + 0.012*"film" + 0.009*"like" + 0.009*"character" + 0.008*"wakanda" + 0.007*"black" + 0.007*"really" + 0.006*"movie" + 0.006*"feel" + 0.005*"the"')
(2, '0.009*"wakanda" + 0.006*"really" + 0.006*"black" + 0.006*"namor" + 0.006*"underwater" + 0.006*"lot" + 0.006*"screen" + 0.003*"i" + 0.003*"movie" + 0.003*"film"')
(3, '0.021*"i" + 0.013*"movie" + 0.012*"the" + 0.009*"film" + 0.008*"like" + 0.007*"also" + 0.007*"character" + 0.007*"marvel" + 0.007*"really" + 0.007*"get"')
(4, '0.019*"i" + 0.009*"movie" + 0.007*"character" + 0.007*"black" + 0.007*"let" + 0.007*"much" + 0.005*"really" + 0.005*"wakanda" + 0.005*"one" + 0.005*"panther"')
Optimal K: 5


## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import LsiModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

# Load the preprocessed data
df = pd.read_csv('Reviews_Analyzed.csv')

# Tokenize the preprocessed text
tokenized_text = [text.split() for text in df['Cleaned_Review']]

# Create a dictionary representation of the documents
dictionary = Dictionary(tokenized_text)

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

# Generate coherence scores for different numbers of topics (K)
def compute_coherence_values(dictionary, corpus, texts, limit):
    coherence_values = []
    for num_topics in range(2, limit + 1):
        # Build LSI model
        lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=num_topics)

        # Compute coherence score
        coherence_model = CoherenceModel(model=lsi_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())

    return coherence_values

# Set the upper limit for the number of topics (K)
limit = 10

# Compute coherence scores for different values of K
coherence_values = compute_coherence_values(dictionary, corpus, tokenized_text, limit)

# Find the optimal number of topics based on the highest coherence score
optimal_k = coherence_values.index(max(coherence_values)) + 2  # Adding 2 because the loop starts from 2
print(f"Optimal number of topics (K): {optimal_k}")

# Build the final LSA model with the optimal number of topics
final_lsa_model = LsiModel(corpus, id2word=dictionary, num_topics=optimal_k)

# Print the topics
topics = final_lsa_model.show_topics(num_words=10)
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")


Optimal number of topics (K): 7
Topic 0: 0.597*"i" + 0.238*"film" + 0.175*"character" + 0.172*"wakanda" + 0.166*"movie" + 0.157*"black" + 0.154*"like" + 0.145*"really" + 0.123*"marvel" + 0.115*"the"
Topic 1: -0.255*"i" + 0.234*"film" + 0.211*"king" + -0.202*"movie" + 0.171*"would" + 0.159*"without" + 0.137*"wakanda" + 0.133*"still" + -0.129*"really" + 0.126*"place"
Topic 2: -0.319*"i" + 0.251*"movie" + 0.234*"wakanda" + 0.187*"forever" + 0.180*"marvel" + 0.160*"namor" + 0.155*"black" + 0.136*"first" + 0.127*"give" + 0.125*"way"
Topic 3: 0.281*"movie" + -0.186*"wakanda" + -0.186*"i" + 0.160*"good" + 0.148*"like" + 0.148*"character" + -0.142*"film" + 0.139*"feel" + -0.126*"one" + -0.116*"forever"
Topic 4: -0.245*"film" + 0.224*"riri" + -0.206*"i" + 0.167*"thought" + 0.141*"still" + 0.138*"mcu" + 0.138*"think" + 0.136*"shuri" + -0.133*"marvel" + 0.128*"wakanda"
Topic 5: 0.253*"one" + -0.233*"phase" + -0.168*"movie" + -0.163*"black" + 0.136*"world" + -0.127*"story" + -0.122*"the" + 0.108*"

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [26]:
pip install lda2vec

  Preparing metadata (setup.py) ... done
  Created wheel for lda2vec: filename=lda2vec-0.16.10-py3-none-any.whl size=14410 sha256=c1c9ffaf32522d3e1d6e7b3baa6cab94cf3caf0e650f3f7e1808ef1d6c467dc9
  Stored in directory: /root/.cache/pip/wheels/1e/90/24/a97126c0fe8b479ba3bb79d3b18ebaab571a18d90bb2967ab6
Successfully built lda2vec


In [24]:
pip install git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis

  Cloning https://github.com/bmabey/pyLDAvis.git (to revision master) to /tmp/pip-install-fmxei4in/pyldavis_d15b35859d3f4be18bd46569bc6fce45
  Running command git clone --filter=blob:none --quiet https://github.com/bmabey/pyLDAvis.git /tmp/pip-install-fmxei4in/pyldavis_d15b35859d3f4be18bd46569bc6fce45
  Resolved https://github.com/bmabey/pyLDAvis.git to commit 997bd7cf26efefd60a5d9915ee22542fd9ca6624
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.2 MB/s eta 0:00:00
  Created wheel for pyLDAvis: filename=pyLDAvis-3.4.1-py3-none-any.whl size=136936 sha256=4604de342588f73304c404c466e4fde2c1a8490cb1aef7e8541c277439c0adc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ljietnbm/wheels/37/51/42/a8bf36e7ac84204262a77656

In [28]:
import pandas as pd
from lda2vec import preprocess, Corpus
from lda2vec.model import LDA2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import matplotlib.pyplot as plt

# Load the preprocessed data
df = pd.read_csv('Reviews_Analyzed.csv')

# Tokenize the preprocessed text
tokenized_text = [text.split() for text in df['Cleaned_Review']]

# Create a dictionary representation of the documents
word_to_id, id_to_word, word_counts = preprocess.tokenize(tokenized_text, min_count=10)

# Convert tokenized text to numerical format
corpus = Corpus()
corpus.update_word_count(word_counts)
corpus.add_documents(tokenized_text, word_to_id)

# Set the range of K
start_k = 2
end_k = 10

# Determine the optimal number of topics (K) based on coherence score
def find_optimal_k(start, end):
    coherence_scores = []
    for k in range(start, end + 1):
        # Build LDA2Vec model
        lda2vec_model = LDA2Vec(corpus, num_topics=k, num_epochs=20)
        lda2vec_model.fit()

        # Get document-topic distribution
        doc_topic_dist = lda2vec_model.get_doc_topic_distribution()

        # Calculate coherence score (you may need to replace this with an appropriate coherence metric)
        coherence_score = np.mean(lda2vec_model.get_coherence(doc_topic_dist))
        coherence_scores.append(coherence_score)

    # Find the optimal K based on the highest coherence score
    optimal_k = coherence_scores.index(max(coherence_scores)) + start
    return optimal_k

# Find the optimal number of topics (K) based on coherence scores
optimal_k = find_optimal_k(start_k, end_k)
print(f"Optimal number of topics (K): {optimal_k}")

# Build LDA2Vec model with the optimal number of topics
final_lda2vec_model = LDA2Vec(corpus, num_topics=optimal_k, num_epochs=20)
final_lda2vec_model.fit()

# Get document-topic distribution
final_doc_topic_dist = final_lda2vec_model.get_doc_topic_distribution()

# Print the top words for each topic
top_words_per_topic = final_lda2vec_model.print_top_words(n_words=10)
for topic_id, words in enumerate(top_words_per_topic):
    print(f"Topic {topic_id}: {', '.join(words)}")


ImportError: ignored

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [1]:
pip install bertopic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 54.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.7 MB/s eta 0:00:00
  Preparing metadata

In [33]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score
import numpy as np

# Load the preprocessed data
df = pd.read_csv('Reviews_Analyzed.csv')

# Ensure 'Cleaned_Review' column is treated as strings
tokenized_text = df['Cleaned_Review'].astype(str).tolist()

# Create a bag-of-words matrix using CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(tokenized_text)

# Reduce dimensionality using Truncated SVD
svd_model = TruncatedSVD(n_components=5)
X_svd = svd_model.fit_transform(X)

# Determine the optimal number of topics (K) based on coherence score
def find_optimal_k(start, end):
    silhouette_scores = []
    for k in range(start, end + 1):
        # Initialize BERTopic model
        model = BERTopic(language="english", calculate_probabilities=True, nr_topics=k)

        # Fit the model
        topics, _ = model.fit_transform(tokenized_text)

        # Get the UMAP embedding
        umap_embedding = model.umap_model.embedding_

        if len(set(topics)) > 1:  # Check if there is more than one topic
            # Calculate the silhouette score as a coherence metric
            silhouette_avg = silhouette_score(umap_embedding, topics)
            silhouette_scores.append(silhouette_avg)
        else:
            silhouette_scores.append(np.nan)

    # Find the optimal K based on the highest silhouette score
    optimal_k = silhouette_scores.index(max(silhouette_scores)) + start
    return optimal_k

# Set the range of K values to explore
start_k = 2
end_k = 10

# Find the optimal number of topics (K) based on silhouette scores
optimal_k = find_optimal_k(start_k, end_k)
print(f"Optimal number of topics (K): {optimal_k}")

# Initialize BERTopic model with the optimal number of topics
final_model = BERTopic(language="english", calculate_probabilities=True, nr_topics=optimal_k)

# Fit the final model
topics, _ = final_model.fit_transform(tokenized_text)

# Print the top words for each topic
top_words = final_model.get_topics()
for topic_id, words in top_words.items():
    print(f"Topic {topic_id}: {', '.join(words)}")


Optimal number of topics (K): 2


TypeError: ignored

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



